# Sensed Energy Error

## Dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# for pipelined data
import emeval.analysed.phone_view as eapv

In [ ]:
import pandas as pd

In [ ]:
import geopandas as gpd
import shapely as shp
import shapely.geometry as shpgeo

In [ ]:
import emeval.viz.geojson as ezgj

In [ ]:
import numpy as np

In [ ]:
import emeval.metrics.dist_calculations as emd

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Analytics results
import emeval.metrics.segmentation as ems

In [ ]:
# for statistics
import scipy as sp
import scipy.stats as spst 

In [ ]:
# For easier debugging while working on modules
import importlib

In [ ]:
import sys 

In [ ]:
sys.path.append('/Users/gkosmach/Documents/every_trip_counts/e-mission-server')

In [ ]:
import emission.analysis.intake.cleaning.location_smoothing as eaicl

In [ ]:
# json
import json

### Load in Phone Views for Server or Pickle File

In [ ]:
import importlib
importlib.reload(eipv)

In [ ]:
# import emission.core.get_database as edb

In [ ]:
# edb.get_timeseries_error_db().find().distinct("metadata.key")
# edb.get_timeseries_db().find().distinct("metadata.key")
# edb.get_timeseries_db().count_documents({"metadata.key": "manual/evaluation_transition"})
# list(edb.get_timeseries_error_db().find({"metadata.key": "manual/evaluation_transition"}).limit(3))
# edb.get_timeseries_error_db().find({"metadata.key": "manual/evaluation_transition"}).distinct("data.transition")
# # list(edb.get_timeseries_db().find({"metadata.key": "manual/evaluation_transition"}).limit(3))
# # edb.get_usercache_db().find({"metadata.key": "manual/evaluation_transition"}).distinct("data.transition")
# # list(edb.get_timeseries_error_db().find({"metadata.key": "manual/evaluation_transition", "data.transition": "START_EVALUATION_PERIOD"}))

In [ ]:
# AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
# # If using ServerSpecDetails, data can alternatively be retrieved as such:
# DATASTORE_LOC = "http://localhost:8080"
# sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")
# pv = eipv.PhoneView(sd)

In [ ]:
def import_sd_and_pv_from_server(trips  = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"], 
                                 AUTHOR_EMAIL  = "shankari@eecs.berkeley.edu", 
                                 DATASTORE_LOC = "http://localhost:8080", 
                                 pkl_file_name = None):
    sd_l = []
    pv_l = []
    for trip in trips:
        sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, trip)
        pv = eipv.PhoneView(sd)
        sd_l.append(sd)
        pv_l.append(pv)
    if pkl_file_name:
        import pickle
        with open(pkl_file_name, 'wb') as outp:
            for pv in pv_l:
                pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)
    return sd_l, pv_l

In [ ]:
def import_pv_from_pkl(pkl_file_name, 
                       trips = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"]):
    import pickle
    pv_l = []
    with open('pv.pkl', 'rb') as inp:
        for trip in trips:
            pv_l.append(pickle.load(inp))
    return pv_l

#### Phone View

In [ ]:
(pv_la, pv_sj, pv_ucb) = import_pv_from_pkl('pv.pkl')

#### Clean View

In [ ]:
av_la = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")
av_sj = eapv.create_analysed_view(pv_sj, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")
av_ucb = eapv.create_analysed_view(pv_ucb, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")

#### GIS view

In [ ]:
gv_la = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
gv_sj = eapv.create_analysed_view(pv_sj, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
gv_ucb = eapv.create_analysed_view(pv_ucb, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

# Temporally Aligned Histories

In [ ]:
import emeval.metrics.reference_trajectory as emr

## Spacio Temporal Trajectories

#### Get histories for phone views

In [ ]:
def get_histories(pv, os, role):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    trips = []
    for phone_os, phone_map in pv.map().items():
        if os != phone_os:
            continue
        for phone_label, phone_detail_map in phone_map.items():
            if "control" in phone_detail_map["role"]:
                continue
            for r_idx, r in enumerate(phone_detail_map["evaluation_ranges"]):
                if r['eval_role_base'] != role:
                    continue
                tr_ss  = []
                tr_gts = []
                for i, tr in enumerate(r["evaluation_trip_ranges"]):
                    for ss in tr["sensed_section_ranges"]:
                        ## s = d / t -> t = d / s
                        ## append the sensed section time and mode data
                        tr_ss.append(ss)
                    for section in tr["evaluation_section_ranges"]:
                        ### begin: get gt time and mode data
                        ## get the ground truth section data
                        gt_leg = pv.spec_details.get_ground_truth_for_leg(
                                                        tr["trip_id_base"], 
                                                        section["trip_id_base"], 
                                                        tr['start_ts'], 
                                                        tr['end_ts']
                                                    )

                        if gt_leg["type"] == "WAITING": 
                            continue
                            
                        gts = {
                            'start_ts': section['start_ts'], 
                            'end_ts': section['end_ts'], 
                            'mode': gt_leg['mode'], 
                            }
                        tr_gts.append(gts)
                        ### end: get gt time and mode data
                # now, we build a timeline for each trip
                trip = tr.copy()
                trip['inf']  = tr_ss
                trip['gt'] = tr_gts
                trip['tz'] = pv.spec_details.eval_tz
                trips.append(trip)
    return trips

In [ ]:
get_histories(gv_la, 'ios', 'HAHFDC');

#### Pad the start and end of a timeline, fill in the transitios with 'ghost' mode

In [ ]:
def pad_and_fill_history(trip):
    h_inf = trip['inf']
    gt_timeline = trip['gt']
    h_inf_padded = []
    h_gt_padded = []
    ####### FILL IN SENSED TIMELINE #######
    ### fill in start ###
    if len(h_inf) == 0:
        if len(gt_timeline) == 0:
            return h_inf, gt_timeline
        else:
            h_inf.append(
                {
                    'mode' : 'NO_SENSED_START',
                    'start_ts' : gt_timeline[0]['start_ts'],
                    'end_ts' : gt_timeline[-1]['end_ts']
                }
            )
    if len(gt_timeline) == 0:
        gt_timeline.append(
            {
                'mode' : 'NO_GT_START',
                'start_ts' : h_inf[0]['start_ts'],
                'end_ts' : h_inf[-1]['end_ts']
            }
        )
    if 'data' in h_inf[0]:
        start_misalignment = h_inf[0]['data']['start_ts'] - gt_timeline[0]['start_ts']
        end_misalignment = h_inf[-1]['data']['end_ts'] - gt_timeline[-1]['end_ts']
    else:
        start_misalignment = h_inf[0]['start_ts'] - gt_timeline[0]['start_ts']
        end_misalignment = h_inf[-1]['end_ts'] - gt_timeline[-1]['end_ts']

    if start_misalignment > 0:
        if 'data' in h_inf[0].keys():
            h_inf[0] = h_inf[0]['data']
        
        h_inf_padded.append(
            {
                'mode' : 'NO_SENSED_START',
                'start_ts' : h_inf[0]['start_ts'] - start_misalignment,
                'end_ts' : h_inf[0]['start_ts']
            }
        )
    ### fill in meat ###
    for ss in h_inf:
        if 'data' in ss.keys():
            ss = ss['data']
        if 'sensed_mode' in ss.keys():
            ss['mode'] = ss['sensed_mode']
        if len(h_inf_padded) > 0:
            ## check to see if there is a gap ##
            if ss['start_ts'] - h_inf_padded[-1]['end_ts'] > 0:
                ## fill in the blank
                h_inf_padded.append(
                    {
                        'mode' : 'NO_SENSED_MIDDLE', 
                        'start_ts' : h_inf_padded[-1]['end_ts'],
                        'end_ts' : ss['start_ts']
                    }
                )
            
        ## the timeline is continuous, and we can fill our section ##
        h_inf_padded.append(ss)
    ### fill in end ###
    if end_misalignment < 0:
        ss = h_inf[-1]
        if 'data' in ss.keys():
            ss = ss['data']
        h_inf_padded.append(
            {
                'mode' : 'NO_SENSED_END',
                'start_ts' : ss['end_ts'],
                'end_ts' : ss['end_ts'] - end_misalignment
            }
        )
    ####### FILL IN GT TIMELINE #######
    ### fill in start ###
    if start_misalignment < 0:
        h_gt_padded.append(
            {
                'mode' : 'NO_GT_START',
                'start_ts' : gt_timeline[0]['start_ts'] + start_misalignment,
                'end_ts' : gt_timeline[0]['start_ts']
            }
        )
    ### fill in meat ###
    for gts in gt_timeline:
        if len(h_gt_padded) > 0:
            ## fill in the blank ##
            if gts['start_ts'] - h_gt_padded[-1]['end_ts'] > 0:
                h_gt_padded.append(
                    {
                        'mode' : 'NO_GT_MIDDLE',
                        'start_ts' : h_gt_padded[-1]['end_ts'],
                        'end_ts' : gts['start_ts']
                    }
                )
        h_gt_padded.append(gts)
    ### fill in end ###
    if end_misalignment > 0:
        h_gt_padded.append(
            {
                'mode' : 'NO_GT_END',
                'start_ts' : h_gt_padded[-1]['end_ts'],
                'end_ts' : h_gt_padded[-1]['end_ts'] + end_misalignment
            }
        )
    return h_inf_padded, h_gt_padded

## Align the histories

In [ ]:
def align_histories(os, role, pv, test=False, test_trip=None):
    if not test:
        if type(pv) is not list: pv = [pv]
        trips = []
        for v in pv:
            trips.extend(get_histories(v, os, role))
    else:
        trips = test_trip if type(test_trip) is list else [test_trip]
    for trip in trips:
        ### Get resampled
        trip_geo_df = emd.to_geo_df(trip['location_df'])
        trip['resampled_df'] = emr.get_int_aligned_trajectory(trip_geo_df, tz=trip['tz'])
        trip['resampled_df'] = eaicl.add_dist_heading_speed(trip['resampled_df'])
        ### Pad and Fill in histories
        H_inf, H_gt = pad_and_fill_history(trip)
        ### Begin Temporal Alignment
        H_alligned= []
        j = 0
        for i in range(len(H_inf)):
            while j < len(H_gt):
                if H_gt[j]['end_ts'] < H_inf[i]['start_ts']:
                    ## get next gt entry
                    j = j + 1
                elif H_inf[i]['end_ts'] > H_gt[j]['start_ts'] and H_inf[i]['start_ts'] < H_gt[j]['end_ts']:
                    h_a = {}
                    start_ts = max(H_gt[j]['start_ts'], H_inf[i]['start_ts'])
                    end_ts   = min(H_gt[j]['end_ts'], H_inf[i]['end_ts'])
                    inf_mode = H_inf[i]['mode']
                    gt_mode  = H_gt[j]['mode']
                    distance = 0
                    interval_df = trip['resampled_df'].query(f"ts >= {start_ts} and ts <= {end_ts}")
                    if interval_df.count().max() > 0:
                        distance = interval_df['distance'].sum()
                    ### create alignment interval
                    h_a = {
                        'start_ts' : start_ts,
                        'end_ts'   : end_ts,
                        'inf_mode' : inf_mode,
                        'gt_mode'  : gt_mode,
                        'duration' : end_ts - start_ts,
                        'distance' : distance
                    }
                    H_alligned.append(h_a)
                    ## get next gt entry
                    j = j + 1
                else: 
                    ## check if next inf anf prev gt entry have overlap
                    if i + 1 < len(H_inf) and j > 0 and H_inf[i+1]['start_ts'] < H_gt[j-1]['end_ts']:
                        ## get prev gt entry
                        j = j-1
                    break
    return H_alligned

### Build Confusion Matrix

In [ ]:
def get_confusion_matrices(os, role, pv):
    return

### Get the sensed/ground-truth trip lengths, making sure that trips do not bleed in to each other

In [ ]:
def get_ss_and_gts_dists(pv_l, os, role):
    if type(pv_l) is not list:
        pv_l = [pv_l]
    trip_dists = []
    for pv in pv_l:
        for phone_os, phone_map in pv.map().items():
            if os != phone_os: continue
            for phone_label, phone_detail_map in phone_map.items():
                for r in phone_detail_map["evaluation_ranges"]:
                    if role not in r['eval_role']: continue
                    if 'control' in r['eval_role']: continue
                    run_ss_dist, run_gt_dist = 0,0
                    for i, tr in enumerate(r["evaluation_trip_ranges"]):
                        sensed_dist, gt_dist = 0,0
                        for ss in tr['sensed_section_ranges']:
                            if 'data' in ss.keys():
                                if i > 0: assert ss['data']['start_ts'] > r["evaluation_trip_ranges"][i-1]['end_ts']
                                if i > 0: assert ss['data']['start_ts'] > trip_dists[-1]['gt_end_ts']
                                sensed_dist += ss['data']['distance']
                            else:
                                sensed_dist = eaicl.add_dist_heading( tr['location_df'] ).distance.sum()
                                break
                        run_ss_dist += sensed_dist
                        for sr in tr['evaluation_section_ranges']:
                            ##### Ground Truth Distance ######
                            gt_leg = pv.spec_details.get_ground_truth_for_leg(
                                tr["trip_id_base"], 
                                sr["trip_id_base"], 
                                tr['start_ts'], 
                                tr['end_ts']
                            )
                            gt_shapes = gpd.GeoSeries(eisd.SpecDetails.get_shapes_for_leg(gt_leg))
                            return gt_shapes
                            if len(gt_shapes) <= 1:
                                continue
                            ## GET THE TOTAL GT DISTANCE OF A SECTION
                            gt_linestring = gt_shapes['route']
                            gt_geo_df = emd.linestring_to_geo_df(gt_linestring)
                            gt_loc_df = emd.to_loc_df(gt_geo_df)
                            gt_loc_with_dist_df =  eaicl.add_dist_heading( gt_loc_df )
                            gt_dist += gt_loc_with_dist_df['distance'].sum()
                        run_gt_dist += gt_dist
                        trip_dists.append(
                            {
                                'sensed_distance' : sensed_dist,
                                'ground_truth_distance' : gt_dist,
                                'gt_end_ts' : tr['evaluation_section_ranges'][-1]['end_ts']
                            }
                        )
    return trip_dists

In [ ]:
importlib.reload(eaicl)

In [ ]:
get_ss_and_gts_dists(pv_la, 'ios', 'HAHFDC')

# CANBIKECO

## Perturb Trip Length

In [ ]:
import scipy as sp
import scipy.stats as spst

In [ ]:
absolute_error_df, relative_error_df = get_approx_err('ios', 'HAHFDC', [av_la, av_sj, av_ucb])

In [ ]:
kernel = spst.gaussian_kde(relative_error_df.to_numpy().flatten())

#### We use the resample function to sample from our kernel for the $n$ needed points

In [ ]:
plt.plot(kernel(100).T)
x = np.linspace(-2,2,300)
y=kernel(x)
plt.plot(x,kernel(x))
plt.plot(x, spst.gaussian_kde(kernel.resample(50))(x))

In [ ]:
import emission.core.get_database as edb
import emission.storage.decorations.trip_queries as esdt

In [ ]:
import importlib

In [ ]:
importlib.reload(edb)

## Get Phone Views from Spec Detials

In [ ]:
def get_phone_views(spec_details):
    if type(spec_details) is not list:
        speci_details = [spec_details]
    phone_view_list = []
    analyzed_view_list = []
    for sd in speci_details:
        pv = eipv.PhoneView(sd)
        phone_view_list.append(pv)
        av = eapv.create_analysed_view(
            pv, 
            sd.DATASTORE_LOC, 
            "analysis/recreated_location", 
            "analysis/cleaned_trip", 
            "analysis/cleaned_section")
        analyzed_view_list.append(av)
    return phone_view_list, analyzed_view_list

## Set Up Error_Rates_db

In [ ]:
analyzed_view_list = [av_la, av_sj, av_ucb]
Error_Dict = {
        "relative_distance_errors" :
        {
            "android:HAMFDC" : np.array(get_approx_err('android', 'HAMFDC', analyzed_view_list)[-1]).tolist(),
            "ios:HAHFDC"     : np.array(get_approx_err('ios', 'HAHFDC', analyzed_view_list)[-1]).tolist()
        }
    }
Error_Rates_db.delete_one({}) ## todo: need to just delete relative distance errors
Error_Rates_db.insert_one(Error_Dict)

## approximation errors

In [ ]:
def get_approx_err(os, role, pv_l):
    trip_dist = get_ss_and_gts_dists(pv_l, os, role)
    relative_error = []
    absolute_error = []
    for i in range(len(trip_dist)):
        abs_err = (trip_dist[i]['sensed_distance'] - trip_dist[i]['ground_truth_distance'])
        rel_err = abs_err / trip_dist[i]['ground_truth_distance']
        relative_error.append(rel_err)
        absolute_error.append(abs_err)
    return absolute_error, relative_error

In [ ]:
def set_Error_Rates_db(spec_details):
    phone_view_list, analyzed_view_list = get_phone_views(spec_details)
    Error_Dict = {
            "relative_distance_errors" :
            {
                "android:HAMFDC" : np.array(get_approx_err('android', 'HAMFDC', analyzed_view_list)[-1]).tolist(),
                "ios:HAHFDC"     : np.array(get_approx_err('ios', 'HAHFDC', analyzed_view_list)[-1]).tolist()
            }
        }
    Error_Rates_db = edb.get_Error_Rates_db()
    try:
        Error_Rates_db.insert_one(Error_Dict)
    except:
        Error_Rates_db.delete_one({}) ## todo: need to just delete relative distance errors
        Error_Rates_db.insert_one(Error_Dict)

## Get Phone OS

In [ ]:
import emission.core.wrapper.user as ecwu

In [ ]:
trips.apply(lambda trip : edb.get_profile_db().find_one({'user_id': trip['user_id']}),axis=1).apply(pd.Series)['curr_platform']

## Add dominant mode and artificial GT to dataframes
These opperations are expensive, and can be done in parrellel

In [ ]:
def get_dominant_mode(s):
    sections = s[0] #need to unpack into list of sections
    i = np.argmax(
        [
            section['data']['duration'] for section in sections
        ]
    ) # get the dominant mode
    return sections[i]['data']['sensed_mode']

In [ ]:
def add_dominant_mode_to_df(trips):
    sections = trips.apply(lambda t : esdt.get_sections_for_trip("analysis/inferred_section", t["user_id"], t['data']['cleaned_trip']), axis=1)
    dominant_mode = sections.to_frame().apply(get_dominant_mode, axis=1)
    trips['data'] = pd.concat([trips['data'].apply(pd.Series), dominant_mode], axis=1).rename({0 : 'dominant_mode'}, axis=1).to_dict('records')
    return trips

In [ ]:
def add_os_to_df(trips):
    try:
        trips['data'].apply(pd.Series).drop('OS', axis=1)
    except:
        pass
    os = trips.apply(lambda trip : edb.get_profile_db().find_one({'user_id': trip['user_id']}),axis=1).apply(pd.Series)['curr_platform']
    trips['data'] = pd.concat([trips['data'].apply(pd.Series), os], axis=1).rename({0 : 'OS'}, axis=1).to_dict('records')
    return trips

In [ ]:
def get_kernels():
    Error_Rates_db = edb.get_Error_Rates_db() ## need to make sure this is filled out
    error_rates_df = pd.DataFrame(edb.get_Error_Rates_db().find({}))
    android_errors = error_rates_df['relative_distance_errors'].apply(pd.Series)['android:HAMFDC'][0]
    ios_errors     = error_rates_df['relative_distance_errors'].apply(pd.Series)['ios:HAHFDC'][0]
    return spst.gaussian_kde(android_errors), spst.gaussian_kde(ios_errors)

#### Find the artificial ground truth for a dataframe

$$
r = \frac{sensed - gt}{gt} \iff sensed = gt(r+1) \iff gt = \frac{sensed}{r+1}
$$

In [ ]:
android_kernel, ios_kernel = get_kernels()
# try: 
#     trips['data'].apply(pd.Series)['OS']
# except:
#     trips = add_os_to_df(trips)
add_dominant_mode_to_df(trips)['data']

In [ ]:
def add_artificial_gt_distance_to_df(trips):
    android_kernel, ios_kernel = get_kernels()
    try: 
        trips['data']['OS']
    except:
        add_os_to_df(trips)
    artificial_gt = np.array(pd.DataFrame(trips)['data'].apply(pd.Series)['distance']) / (np.array(kernel.resample(len(pd.DataFrame(trips)))) + 1)
    trips['data'] = pd.concat([trips['data'].apply(pd.Series), pd.Series(artificial_gt.flatten())], axis=1).rename({0 : 'artificial_gt_distance'}, axis=1).to_dict('records')
    return trips

In [ ]:
upper_limit = 10
trips = [doc for doc in Stage_analysis_timeseries.find({"metadata.key":"analysis/confirmed_trip"})]
trips = pd.DataFrame(trips[:upper_limit])
sections = trips.apply(lambda t : esdt.get_sections_for_trip("analysis/inferred_section", t["user_id"], t['data']['cleaned_trip']), axis=1)
dominant_mode = sections.to_frame().apply(get_dominant_mode, axis=1)
trips['data'] = pd.concat([trips['data'].apply(pd.Series), dominant_mode], axis=1).rename({0 : 'dominant_mode'}, axis=1).to_dict('records')
artificial_gt = np.array(pd.DataFrame(trips)['data'].apply(pd.Series)['distance']) / (np.array(kernel.resample(len(pd.DataFrame(trips)))) + 1)
trips['data'] = pd.concat([trips['data'].apply(pd.Series), pd.Series(artificial_gt.flatten())], axis=1).rename({0 : 'artificial_gt_distance'}, axis=1).to_dict('records')
trips

## Get the artificial GT distance

In [ ]:
artificial_gt = np.array(pd.DataFrame(confirmed_trips)['data'].apply(pd.Series)['distance']) / (np.array(kernel.resample(len(pd.DataFrame(confirmed_trips)))) + 1)

In [ ]:
df = pd.DataFrame(confirmed_trips)
df['data'].apply(pd.Series).columns


#.inferred_labels.to_frame()[df['data'].apply(pd.Series)#.inferred_labels.to_frame().astype(bool)].query('inferred_labels == inferred_labels')['inferred_labels'].apply(pd.Series)